In [1]:
import csv
import networkx as nx
import ast

In [2]:
def flatten_list(l):
    vals = []
    for el in l:
        if type(el)==list:
            vals += el
        else:
            vals.append(el)
    if [e for e in vals if type(e)==list]:
        return flatten_1ist(vals)
    else:
        return vals

In [3]:
with open('stn_edge_list_attrs.csv', 'r') as f:
#     WOS:A1979JC64200002
#     WOS:A1994QB12400003
    reader = csv.DictReader(f)
    print(reader.fieldnames)
    rows = [row for row in reader]
len(rows)

['citer', 'citee', 'year', 'journal', 'article_id', 'language', 'keywords']


854438

In [4]:
name_equivalences = []
with open('equivalent_names.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        name_equivalences.append((row['name'], row['canonical_name']))
canonical_names = []
with open('phil_list_identifiers.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        canonical_names.append((row['identifier'], row['yob']))
# print(canonical_names)
# print(name_equivalences)
nodes_to_swap = [neq[0] for neq in name_equivalences]
# need to check nodes and edges against this first
# for nodes if they're in nodes to swap just remove them from the list
# for edges if the first element is in nodes_to_swap, then replace it with the second
can_node_names = [n[0] for n in canonical_names]
swap_dict = {neq[0]: neq[1] for neq in name_equivalences}

In [5]:
edges = []
for row in rows:
#   I need to treat the string here as a list!
    keywords = flatten_list(ast.literal_eval(row['keywords']))
    year = int(row['year'])
    citer = row['citer']
    if row['citer'] in nodes_to_swap:
        citer = swap_dict[citer]
    citee = row['citee']
    if row['citee'] in nodes_to_swap:
        citee = swap_dict[citee]
    journal = row['journal']
    lang = row['language']
    article_id = row['article_id']
    edges.append((citer, citee, {'year': year, 'journal': journal,
                                 'article_id': article_id, 'lang': lang,
                                 'keywords': keywords}))

In [6]:
G = nx.DiGraph()
G.add_edges_from(edges)

In [7]:
yobs_dict = {cn[0]: int(cn[1]) for cn in canonical_names}
names = yobs_dict.keys()
nodes = G.nodes()
drop_list = []
for name in names:
    if name not in nodes:
        drop_list.append(name)
for name in drop_list:
    yobs_dict.pop(name, None)
yobs_dict
yobs_dict['knobe j'] = 1974

In [8]:
nx.set_node_attributes(G, 'yob', yobs_dict)
[n for n in G.nodes(data=True) if 'yob' in n[1].keys()]

[('torretti r', {'yob': 1930}),
 ('bauer b', {'yob': 1809}),
 ('alston w', {'yob': 1921}),
 ('pogge t', {'yob': 1953}),
 ('aron r', {'yob': 1905}),
 ('campbell g', {'yob': 1719}),
 ('ong w j', {'yob': 1912}),
 ('biran m d', {'yob': 1766}),
 ('bouveresse j', {'yob': 1940}),
 ('edie j m', {'yob': 1927}),
 ('etchemendy j', {'yob': 1952}),
 ('hardt m', {'yob': 1960}),
 ('certeau m d', {'yob': 1925}),
 ('lewis c i', {'yob': 1883}),
 ('rougier l', {'yob': 1889}),
 ('lipps t', {'yob': 1851}),
 ('price h h', {'yob': 1899}),
 ('ficino m', {'yob': 1433}),
 ('ramsey f p', {'yob': 1903}),
 ('lepore e', {'yob': 1950}),
 ('albritton r', {'yob': 1923}),
 ('leibniz g w v', {'yob': 1646}),
 ('lynch m', {'yob': 1942}),
 ('gall f j', {'yob': 1758}),
 ('bacon f', {'yob': 1561}),
 ('crathorn w', {'yob': 1938}),
 ('ilyenkov e', {'yob': 1924}),
 ('knasas j f x', {'yob': 1948}),
 ('arendt h', {'yob': 1906}),
 ('montague r', {'yob': 1930}),
 ('thagard p', {'yob': 1950}),
 ('burtt e a', {'yob': 1892}),
 ('railt

In [22]:
not_canonical = {n[0]: 0 for n in G.nodes(data=True) if 'yob' not in n[1] or 'yob' in n[1] and n[1]['yob'] >= 1910}
canonical = {n[0]: 1 for n in G.nodes(data=True) if 'yob' in n[1] and n[1]['yob'] < 1910}

canonical.update(not_canonical)

In [23]:
nx.set_node_attributes(G, 'bipartite', canonical)

In [24]:
[n for n in G.nodes(data=True) if n[1]['bipartite']==1]

[('bauer b', {'bipartite': 1, 'yob': 1809}),
 ('aron r', {'bipartite': 1, 'yob': 1905}),
 ('campbell g', {'bipartite': 1, 'yob': 1719}),
 ('biran m d', {'bipartite': 1, 'yob': 1766}),
 ('lewis c i', {'bipartite': 1, 'yob': 1883}),
 ('rougier l', {'bipartite': 1, 'yob': 1889}),
 ('lipps t', {'bipartite': 1, 'yob': 1851}),
 ('price h h', {'bipartite': 1, 'yob': 1899}),
 ('ficino m', {'bipartite': 1, 'yob': 1433}),
 ('ramsey f p', {'bipartite': 1, 'yob': 1903}),
 ('leibniz g w v', {'bipartite': 1, 'yob': 1646}),
 ('gall f j', {'bipartite': 1, 'yob': 1758}),
 ('bacon f', {'bipartite': 1, 'yob': 1561}),
 ('arendt h', {'bipartite': 1, 'yob': 1906}),
 ('burtt e a', {'bipartite': 1, 'yob': 1892}),
 ('romero f', {'bipartite': 1, 'yob': 1891}),
 ('vattel e d', {'bipartite': 1, 'yob': 1714}),
 ('boethius', {'bipartite': 1, 'yob': 480}),
 ('mendelssohn m', {'bipartite': 1, 'yob': 1729}),
 ('waddington c h', {'bipartite': 1, 'yob': 1905}),
 ('chardin p t d', {'bipartite': 1, 'yob': 1881}),
 ('schol

In [25]:
nx.write_gpickle(G,'bipartite_pickle.pickle',2)

In [49]:
# in_degs, out_degs, degs = G.in_degree(), G.out_degree(), G.degree()
# nx.set_node_attributes(G, 'in_degree', in_degs)
# nx.set_node_attributes(G, 'out_degree', out_degs)
# nx.set_node_attributes(G, 'degree', degs)

In [27]:
sorted([n for n in G.nodes(data=True) if n[1]['bipartite']])

[('aaron r', {'bipartite': 1, 'yob': 1901}),
 ('adler m j', {'bipartite': 1, 'yob': 1902}),
 ('adorno t w', {'bipartite': 1, 'yob': 1903}),
 ('al-farabi', {'bipartite': 1, 'yob': 872}),
 ('al-ghazali', {'bipartite': 1, 'yob': 1058}),
 ('al-jawziyya i q', {'bipartite': 1, 'yob': 1292}),
 ('alexander s', {'bipartite': 1, 'yob': 1859}),
 ('alhazen', {'bipartite': 1, 'yob': 965}),
 ('anaxagoras', {'bipartite': 1, 'yob': -495}),
 ('anderson j', {'bipartite': 1, 'yob': 1893}),
 ('antisthenes', {'bipartite': 1, 'yob': -443}),
 ('apuleius', {'bipartite': 1, 'yob': 125}),
 ('aquinas t', {'bipartite': 1, 'yob': 1225}),
 ('arabi i', {'bipartite': 1, 'yob': 1165}),
 ('archimedes', {'bipartite': 1, 'yob': -286}),
 ('arendt h', {'bipartite': 1, 'yob': 1906}),
 ('aristotle', {'bipartite': 1, 'yob': -383}),
 ('arnauld a', {'bipartite': 1, 'yob': 1612}),
 ('aron r', {'bipartite': 1, 'yob': 1905}),
 ('aurelius m', {'bipartite': 1, 'yob': 121}),
 ('aurobindo s', {'bipartite': 1, 'yob': 1872}),
 ('avenari

In [58]:
[e for e in G.edges(data=True) if e[0]=='weber m']

[('weber m',
  'schrader-frechette k',
  {'article_id': 'WOS:000079574800003',
   'journal': 'PHILOS SCI',
   'keywords': ['SPECIES-DIVERSITY', ' COMPETITION'],
   'lang': 'English',
   'year': 1999}),
 ('weber m',
  'graves l',
  {'article_id': 'WOS:000244719000002',
   'journal': 'PHILOS SCI',
   'keywords': ['EVOLUTIONARY-THEORY',
    ' STATISTICAL CHARACTER',
    ' QUANTUM-MECHANICS',
    '',
    'ACTION-POTENTIALS',
    ' CHANNEL',
    ' PROBABILITY',
    ' BIOLOGY',
    ' MODEL'],
   'lang': 'English',
   'year': 2005}),
 ('weber m',
  'gibbard a',
  {'article_id': 'WOS:000076672800006',
   'journal': 'ETHICS',
   'keywords': ['DECISION-MAKING',
    ' REGRET',
    ' UNCERTAINTY',
    ' DISAPPOINTMENT',
    ' CHOICE',
    ' RISK'],
   'lang': 'English',
   'year': 1998}),
 ('weber m',
  'railton p',
  {'article_id': 'WOS:000076672800006',
   'journal': 'ETHICS',
   'keywords': ['DECISION-MAKING',
    ' REGRET',
    ' UNCERTAINTY',
    ' DISAPPOINTMENT',
    ' CHOICE',
    ' RISK']

In [39]:
new_list = []
def flatten(alist):
    vals = []
    for el in alist:
        while type(el) == list:
            
flatten(['a','b',['b','c']])

IndentationError: expected an indented block (<ipython-input-39-9713da10d457>, line 7)

In [37]:
# l = ['a', 'b', ['b', [['cds']]]]
l =   [['SPATIAL MEMORY', ' MOBILE ROBOTS', ' NAVIGATION', ' REPRESENTATIONS', ''], ['CONNECTIONISM', ' IMAGERY', ' MODEL']]
vals = []


flatten_list(l)

['SPATIAL MEMORY',
 ' MOBILE ROBOTS',
 ' NAVIGATION',
 ' REPRESENTATIONS',
 '',
 'CONNECTIONISM',
 ' IMAGERY',
 ' MODEL']

In [14]:
help(filter)

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.

